### Примеры использования обёртки для simplekml - библиотеки для формирования KML-файлов (https://simplekml.readthedocs.io/en/latest/index.html)

In [1]:
# Для доступа к пакету из любых ноутбуков/скриптов нужно скопировать папку Tele2_BDA в директорию:
# PYTHON_INTERPRETER_FOLDER_PATH\Lib\site-packages
# Например: C:\Programs\Miniconda3\envs\geoenv\Lib\site-packages\Tele2_BDA\wrappers
from Tele2_BDA.wrappers import fast_tdsql as td
from Tele2_BDA.wrappers import fast_kml
import os
import pandas as pd

#### Папка для хранения демо файлов

In [2]:
dir_name = 'fast_kml_demo_data'
if not os.path.exists(dir_name):
    os.makedirs(dir_name)
os.chdir(dir_name)

#### Установка соединения с Teradata

In [3]:
con = td.set_connection('USER', 'PASSWORD')

#### Отрисовка слоя с точками

In [4]:
q = \
"""
SELECT *
FROM uat_dm.gi_geo_samples
WHERE id IN (1, 4)
ORDER BY id
"""

In [5]:
%%time
points_df = td.select(con, q)

(2, 3)
Wall time: 91.7 ms


In [6]:
points_df.head()

,ID,DESCRIPTION,WKT
0,1,sample_point,POINT (37.642179312454687 55.756365925177171)
1,4,sample_multipoint,MULTIPOINT (37.618575873123632 55.736462051322...


In [7]:
fast_kml.points_kml(points_df, 'points.kml', wkt_column='WKT', name_column='ID')

#### Отрисовка слоя с линиями без описаний

In [8]:
q = \
"""
SELECT *
FROM uat_dm.gi_geo_samples
WHERE id IN (2, 5)
ORDER BY id
"""

In [9]:
%%time
lines_df = td.select(con, q)

(2, 3)
Wall time: 57.9 ms


In [10]:
lines_df.head()

,ID,DESCRIPTION,WKT
0,2,sample_linestring,LINESTRING (37.608791174637304 55.749168851564...
1,5,sample_multilinestring,MULTILINESTRING ((37.608791174637304 55.749168...


In [11]:
fast_kml.lines_kml(lines_df, 'lines.kml', wkt_column='WKT', name_column='ID', description_columns=None)

#### Отрисовка слоя с полигонами случайной расцветки 

In [12]:
q = \
"""
SELECT *
FROM uat_dm.gi_geo_samples
WHERE id IN (3, 6, 7, 8)
ORDER BY id
"""

In [13]:
%%time
polys_df = td.select(con, q)

(4, 3)
Wall time: 116 ms


In [14]:
polys_df.head()

,ID,DESCRIPTION,WKT
0,3,sample_polygon,POLYGON ((37.608962836014257 55.74921715849851...
1,6,sample_multipolygon,MULTIPOLYGON (((37.608791174637304 55.73902307...
2,7,sample_polygon_with_inner_ring,POLYGON ((37.618575873123632 55.74501423149325...
3,8,sample_multipolygon_with_inner_ring,MULTIPOLYGON (((37.608791174637304 55.73902307...


In [15]:
fast_kml.polygons_kml(polys_df, 'polys.kml', wkt_column='WKT', name_column='ID', color_mode='random')

#### Отрисовка слоя с полигонами из текста (для быстрого просмотра геометрий на карте аналогично https://arthur-e.github.io/Wicket/sandbox-gmaps3.html)

In [16]:
# в отличие от Wicket, можно одновременно отрисовать несколько одинаковых геометрий
geodata = [
    'POLYGON((37.59797650788926 55.739627059610996,37.602997603165136 55.739627059610996,37.602997603165136 55.73667950285141,37.59797650788926 55.73667950285141,37.59797650788926 55.739627059610996))',
    'POLYGON((37.59046632264756 55.73977201567571,37.5948866031041 55.74047262906507,37.590809645401464 55.737041919373446,37.58711892579697 55.74027935766223,37.59046632264756 55.73977201567571))'
]
text_polys_df = pd.DataFrame({'wkt': geodata})

In [17]:
text_polys_df.head()

,wkt
0,"POLYGON((37.59797650788926 55.739627059610996,..."
1,"POLYGON((37.59046632264756 55.73977201567571,3..."


In [18]:
# можно не задавать явное название для колонок с геометриями и с названиями
# в качетсве геометрий будет использована последняя колонка в датафрейме
# в качестве названий - индекс датафрейма
# также можно не указывать расширение для файла
fast_kml.polygons_kml(text_polys_df, 'text_polys_df')

#### Закрытие подключения к бд

In [19]:
con.close()